# Environment Configuration

In [ ]:
!pip install --upgrade gspread google-auth

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
import numpy as np
from datetime import datetime

from google.colab import auth
from google.cloud import bigquery
from gspread_dataframe import set_with_dataframe

import gspread
from oauth2client.client import GoogleCredentials
from google.auth import default
creds, _ = default()

import pandas_gbq
from tqdm import tqdm
import time
from datetime import datetime
import time
import random
import json
import requests
import random
from datetime import datetime
import pytz

auth.authenticate_user()
print('Authenticated')

# Project Configuration
project_id = 'foodpanda-tw-bigquery'
client = bigquery.Client(project=project_id)

In [ ]:
# Initialize timezone for Taiwan
taiwan_tz = pytz.timezone('Asia/Taipei')

In [ ]:
# Get current date and time in Taiwan Time
start_datetime = datetime.now(taiwan_tz)
start_time = time.time()

In [ ]:
# Print the start date and time
print(f"Script started at: {start_datetime.strftime('%Y-%m-%d %H:%M:%S %Z')}")

# Open Route API Test Area

In [ ]:
api_key = "open_route_api_key"

coordinates = [[121.2959933,25.009717], [121.2572352,25.0022033]]

url = "https://api.openrouteservice.org/v2/directions/cycling-electric"

payload = {
    "coordinates": coordinates,
    "units": "m"
}

headers = {
    "Authorization": api_key,
    "Content-Type": "application/json"
}

response = requests.post(url, json=payload, headers=headers)

data = response.json()

print(data)

duration_seconds = data['routes'][0]['summary']['duration']
distance_meters = data['routes'][0]['summary']['distance']

duration_minutes = duration_seconds / 60

print(f"兩點之間使用電動腳踏車的交通時間是: {duration_minutes} 分鐘, 路徑是 {distance_meters} 公尺")


# [Main Script] Drive time for UE McDonlad Delivery Area

In [ ]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default

# 授權Google Sheets API訪問
creds, _ = default()
gc = gspread.authorize(creds)

In [ ]:
# 使用Google Sheet的URL或者名稱
spreadsheet_url = 'https://docs.google.com/spreadsheets/link'

# 打開Google Sheet
spreadsheet = gc.open_by_url(spreadsheet_url)

# 選擇你想要操作的工作表
worksheet = spreadsheet.worksheet("01")

In [ ]:
# 讀取工作表的所有數據
rows = worksheet.get_all_values()

# 將數據轉換為Pandas DataFrame
import pandas as pd
df = pd.DataFrame(rows[1:], columns=rows[0])  # 第一行是列名

# 顯示數據
df

In [ ]:
# 將指定的欄位轉換為浮點數類型
df['latitude'] = df['latitude'].astype(float)
df['longitude'] = df['longitude'].astype(float)
df['scraping_latitude'] = df['scraping_latitude'].astype(float)
df['scraping_longitude'] = df['scraping_longitude'].astype(float)

In [ ]:
# drive_time & distance

api_key = "open_route_api_key"

def calculate_drive_time_and_distance(vendor_lat, vendor_lon, scraping_lat, scraping_lon):
    coordinates = [[vendor_lon, vendor_lat], [scraping_lon, scraping_lat]]

    url = "https://api.openrouteservice.org/v2/directions/cycling-electric"

    payload = {
        "coordinates": coordinates,
        "units": "m"
    }

    headers = {
        "Authorization": api_key,
        "Content-Type": "application/json"
    }

    response = requests.post(url, json=payload, headers=headers)

    if response.status_code == 200:
        data = response.json()
        duration_seconds = data['routes'][0]['summary']['duration']
        distance_meters = data['routes'][0]['summary']['distance']

        # Calculate drive time in minutes
        drive_time = duration_seconds / 60
        return drive_time, distance_meters
    else:
        print(f"Error: {response.status_code}")
        return None, None

# Initialize lists to store drive_time and distance
drive_times = []
distances = []

for index, row in df.iterrows():
    drive_time, distance = calculate_drive_time_and_distance(
        row['latitude'], row['longitude'],
        row['scraping_latitude'], row['scraping_longitude']
    )

    drive_times.append(drive_time)
    distances.append(distance)

    time.sleep(1.5)  # To avoid hitting the rate limit

# Add drive_time and distance to the DataFrame
df['drive_time'] = drive_times
df['distance'] = distances


In [ ]:
df

In [ ]:
# 將結果儲存回CSV檔案
df.to_csv('/content/drive/MyDrive/aaa.csv', index=False)

print("計算完成，已將結果儲存到新的CSV檔案。")

In [ ]:
import pandas as pd
import gspread
from google.colab import auth
from google.auth import default

# 授權並連接到 Google Sheets API
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

# 打開Google Sheet
spreadsheet_url = 'https://docs.google.com/spreadsheets/link'
spreadsheet = gc.open_by_url(spreadsheet_url)

# 選擇你要覆寫的工作表 "02"
worksheet_02 = spreadsheet.worksheet("02")

# 生成你的DataFrame (假設 df 是你的結果表)
# 例如：這裡是你前面的df
data = [df.columns.values.tolist()] + df.values.tolist()

# 清空工作表（選擇性步驟）
worksheet_02.clear()

# 將數據寫入到工作表 "02"，從A1單元格開始
worksheet_02.update('A1', data)

print("Data has been overwritten to sheet '02'.")


In [ ]:
end_time = time.time()
execution_time = end_time - start_time
print(f"Total execution time: {execution_time:.2f} seconds")